# 교차 검증과 그리드 서치

## 검증 세트

In [2]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

In [4]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [6]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [7]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증

In [8]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.0130465 , 0.01138473, 0.00644231, 0.00637412, 0.00600767]), 'score_time': array([0.00295115, 0.00158   , 0.00256848, 0.00175333, 0.00196171]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

In [12]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [13]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [14]:
gs.fit(train_input, train_target)

,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'min_impurity_decrease': [0.0001, 0.0002, ...]}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [15]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [16]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [17]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [18]:
print(gs.cv_results_['params'][gs.best_index_])

{'min_impurity_decrease': 0.0001}


In [19]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'max_depth': range(5, 20), 'min_impurity_decrease': array([0.0001... 0.0009]), 'min_samples_split': range(2, 100, 10)}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [21]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [22]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

In [23]:
from scipy.stats import uniform, randint

In [24]:
rgen = randint(0, 10)
rgen.rvs(10)

array([5, 7, 5, 7, 9, 9, 3, 8, 4, 3])

In [25]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98, 101,  95, 105,  98, 112,  84, 102,  97, 108]))

In [26]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.03238404, 0.46721717, 0.24133187, 0.45068925, 0.7733876 ,
       0.2950434 , 0.72124335, 0.65596885, 0.11310964, 0.76156924])

In [27]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [28]:
from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)

,estimator,DecisionTreeC...ndom_state=42)
,param_distributions,"{'max_depth': <scipy.stats....x7c1f78cf3fd0>, 'min_impurity_decrease': <scipy.stats....x7c1f7432e7d0>, 'min_samples_leaf': <scipy.stats....x7c1f7317d1e0>, 'min_samples_split': <scipy.stats....x7c1f7432eb30>}"
,n_iter,100
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [29]:
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


In [30]:
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [31]:
dt = rs.best_estimator_

print(dt.score(test_input, test_target))

0.86


- 검증 세트
    - 하이퍼파라미터 튜닝을 위해 모델을 평가할 때, 테스트 세트를 사용하지 않기 위해 훈련 세트에서 다시 떼어 낸 데이터 세트
- 교차 검증
    - 훈련 세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트의 역할을 하고 나머지 폴드에서는 모델을 훈련
    - 모든 폴드에 대해 검증 점수를 얻어 평균하는 방법
- 그리드 서치
    - 하이퍼파라미터 탐색을 자동화 해주는 도구
    - 탐색할 매개변수를 나열하면 교차 검증을 수행하여 가장 좋은 검증 점수의 매개변수 조합을 선택
    - 마지막으로 이 매개변수 조합으로 최종 모델을 훈련
- 랜덤 서치
    - 연속된 매개변수 값을 탐색할 때 유용함
    - 탐색할 값을 직접 나열하는 것이 아니고 탐색 값을 샘플링할 수 있는 확률 분포 객체를 전달
    - 지정된 횟수만큼 샘플링하여 교차 검증을 수행하기 때문에 시스템 자원이 허락하는 만큼 탐색량을 조절할 수 있음

### 핵심 패키지와 함수
- scikit-learn
    - cross_validate()
        - 교차 검증을 수행하는 함수
        - 첫번째 매개 변수에 교차 검증을 수행할 모델 객체를 전달함.
        - 두 번째와 세 번째 매개변수에 특성과 타깃 데이터를 전달함.
        - scoring 매개 변수
            - 검증에 사용할 평가 지표에 지정할 수 있음.
            - 기본적으로 분류 모델은 정확도를 의미하는 'accuracy'
            - 회귀 모델은 결정 계수를 의미하는 'r2'
        - cv 매개변수
            - 교차 검증 폴드 수나 분할기 객체를 지정할 수 있음.
            - 기본값은 5
            - 회귀일 때는 KFold 클래스를 사용
            - 분류일 때는 StratifiedkFold 클래스를 사용
                - 5-폴드 교차검증 수행
        - n_jobs 매개변수
            - 교차 검증을 수행 할 때 사용할 CPU 코어 수를 지정
            - 기본값은 1
                - 하나의 코어를 사용
            - -1로 지정하면 시스 템에 있는 모든 코어를 사용
        - return_train_score 매개변수
            - True로 지정하면 훈련 세트의 점수도 반환함
            - 기본값은 False
    - GridSearchCV
        - 교차 검증으로 하이퍼파라미터 탐색을 수행
        - 최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델 훈련
        - 첫 번째 매개변수로 그리드 서치를 수행할 모델 객체를 전달
        - 두 번째 매개변수에는 탐색할 모델의 매개변수와 값을 전달
        - scoring, cv, n_jobs, return_train_score 매개변수
            - cross_validate() 함수와 동일
    - RandomizedSearchCV
        - 교차 검증으로 랜덤한 하이퍼파라미터 탐색을 수행
        - 최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련
        - 첫 번째 매개변수로 그리드 서치를 수행할 모델 객체를 전달
        - 두 번째 매개변수에는 탐색할 모델의 매개변수와 확률 분포 객체를 전달
        - scoring, cv, n_jobs, return_train_score 매개변수
            - cross_validate() 함수와 동일

### 확인문제
1. 훈련 세트를 여러 개의 폴드로 나누고 폴드 1개는 평가 용도로, 나머지 폴드는 훈련 용도로 사용합니다. 그다음 모든 폴드를 평가 용도로 사용하게끔 폴드 개수만큼 이 과정을 반복합니다. 이런 평가 방법을 무엇이라고 부르나요?
    1. 교차 검증
    2. 반복 검증
    3. 교차 평가
    4. 반복 평가
    > 정답: 1

2. 다음 중 교차 검증을 수행하지 않는 함수가 클래스는 무엇인가요?
    1. cross_validata()
    2. GridSearchCV
    3. RandomizedSearchCV
    4. train_test_split
    > 정답: 4

3. 다음 중 GridSearchCV에 대해 올바르게 설명한 것은 무엇인가요?
    1. 사람의 개입없이 하이퍼파라미터를 자동으로 검색합니다.
    2. 확률 분포에서 후보 매개변수를 샘플링합니다.
    3. 기본적으로 5-폴드 교차 검증을 수행하여 최적의 하이퍼파라미터 조합을 찾습니다.
    4. DecisionTreeClassifier만 사용할 수 있습니다.
    > 정답: 3

4. 마지막 RandomizedSearchCV 예제에서 DecisionTreeClassifier 클래스에 splitter='random' 매개변수를 추가하고 다시 훈련해 보세요. splitter 매개변수의 기본값은 'best'로 각 노드에서 최선의 분할을 찾습니다. 'random'이면 무작위로 분할한 다음 가장 좋은 것을 고릅니다. 왜 이런 옵션이 필요한지는 다음 절에서 알 수 있습니다. 테스트에서 성능이 올라갔나요 내려갔나요?

In [32]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

,estimator,DecisionTreeC...tter='random')
,param_distributions,"{'max_depth': <scipy.stats....x7c1f78cf3fd0>, 'min_impurity_decrease': <scipy.stats....x7c1f7432e7d0>, 'min_samples_leaf': <scipy.stats....x7c1f7317d1e0>, 'min_samples_split': <scipy.stats....x7c1f7432eb30>}"
,n_iter,100
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [33]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': np.float64(0.00011407982271508446), 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077


둘다 모두 조금 낮음.
최적의 매개 변수 조합을 찾지 못함.